In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
#file_id = '1J05tsCBwdhEwkXygcbpKNztsXA6qErPC'
#downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
# !pip install twitterscraper
# !pip install vaderSentiment
# !pip install langdetect

In [ ]:
#Importing the relevent libraries
import pandas as pd 
import datetime as dt 
from twitterscraper import query_tweets
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langdetect import detect 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.mlab as mlab


In [ ]:
begin_date = dt.date(2018,1,1)
end_date = dt.date(2020,3,26)
limit = 100000
lang = 'english'


In [ ]:
# from google.colab import drive
# drive.mount('/gdrive/')
# %cd /gdrive/My Drive/Morgan Stanley Project/Data and Notebook/
#!cat '/gdrive/My Drive/Morgan Stanley Project/Data and Notebook/pretty_ms_tweets.json'

In [ ]:
#tweets = query_tweets("morganstanley", begindate = begin_date, enddate = end_date, limit = limit, lang = lang)

In [ ]:
#Transforming it into a dataframe
#df = pd.DataFrame(t.__dict__ for t in tweets)
#Saving a file
#df.to_csv('TwitterData.csv')

In [ ]:
df = pd.read_json('pretty_ms_tweets.json')

In [ ]:
#Sentiment Analysis
#Vader Sentiment

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
sentiment = df['text'].apply(lambda x: analyzer.polarity_scores(x))

In [ ]:
#Inserting the sentiment into the dataframe
df = pd.concat([df, sentiment.apply(pd.Series)],1)

In [ ]:
#df['compound'].hist()
#df['neg'].hist()
#df['neu'].hist()
#df['pos'].hist()

In [ ]:
df['date'] = df['timestamp'].apply(lambda x: x.date())

In [ ]:
#Plotting the change in sentiment over time
plt.figure(figsize = (20,6))
plt.plot(df.groupby('date').mean()['compound'].index, df.groupby('date').mean()['compound'].values, label = 'com')

plt.plot(df.groupby('date').mean()['neg'].index, df.groupby('date').mean()['neg'].values)
plt.plot(df.groupby('date').mean()['neu'].index, df.groupby('date').mean()['neu'].values)
plt.plot(df.groupby('date').mean()['pos'].index, df.groupby('date').mean()['pos'].values)

In [ ]:
df['text'].iloc[0]
#As we can see, several of the tweets captures donot refer to morgan stanley. Hence we need to filter out these tweets

In [ ]:
Filter = df['text'].apply(lambda x: 'morganstanley' in x.lower())
df_filter = df[Filter]
df_filter.shape
df_filter['text'].iloc[0]

In [ ]:
plt.figure(figsize = (20,6))
plt.plot(df_filter.groupby('date').mean()['compound'].index, df_filter.groupby('date').mean()['compound'].values, label = 'Compound')
plt.plot(df_filter.groupby('date').mean()['neg'].index, df_filter.groupby('date').mean()['neg'].values, label = 'Negative')
plt.plot(df_filter.groupby('date').mean()['neu'].index, df_filter.groupby('date').mean()['neu'].values, label = 'Neutral')
plt.plot(df_filter.groupby('date').mean()['pos'].index, df_filter.groupby('date').mean()['pos'].values, label = 'Positive')

plt.legend(loc = 'best')
plt.title('Sentiment over Time')
plt.xlabel('Date')
plt.ylabel('Sentiment')

In [ ]:

plt.hist(df_filter['compound'].values, label = 'Compound')
plt.hist(df_filter['neg'].values, label = 'Negative')
plt.hist(df_filter['neu'].values, label = 'Neutral')
plt.hist(df_filter['pos'].values, label = 'Positive')

plt.legend(loc = 'best')

In [ ]:
df['compound'].hist()
#df['neg'].hist()
#df['neu'].hist()
#df['pos'].hist()

In [ ]:
df.iloc[:,-4:-1]

In [ ]:
#Importing Various twitter data
#Merril Lynch
df_ml = pd.read_json('pretty_ml_tweets.json')

#Goldman Sachs
df_gs = pd.read_json('pretty_gs_tweets.json')

#Morgan Stanley
df_ms = pd.read_json('pretty_ms_tweets.json')

#JP Morgan and Chase
df_jp = pd.read_json('pretty_jp_tweets.json')

#Wealth Management
df_wm = pd.read_json('wm_tweets.json')

In [ ]:
df_all = pd.concat([df_ml,df_gs,df_ms,df_jp,df_wm])

In [ ]:
df_all.count()

In [ ]:
#LDAVIS
# !pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf = tf_vectorizer.fit_transform(df['text'].values)
print(dtm_tf.shape)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())

In [ ]:
#dtm_tfidf = tfidf_vectorizer.fit_transform(df_all['text'].values)
#lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
#lda_tfidf.fit(dtm_tfidf)
#pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [ ]:
sentiment_wm = df_wm['text'].apply(lambda x: analyzer.polarity_scores(x))
df_wm = pd.concat([df_wm, sentiment_wm.apply(pd.Series)],1)

In [ ]:
df_wm.head()

In [ ]:
df_wm_pos = df_wm[df_wm['compound'] >= 0.05]
df_wm_neg = df_wm[df_wm['compound'] <= -0.05]
df_wm_neu = df_wm[( df_wm['compound'] < 0.05 ) & ( df_wm['compound'] > -0.05 )]


In [ ]:
dtm_tfidf = tfidf_vectorizer.fit_transform(df_wm_pos['text'].values)
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [ ]:
dtm_tfidf = tfidf_vectorizer.fit_transform(df_wm_neg['text'].values)
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [ ]:
dtm_tfidf = tfidf_vectorizer.fit_transform(df_wm_neu['text'].values)
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [ ]:
print(df_wm_pos.shape)
print(df_wm_neg.shape)
print(df_wm_neu.shape)

In [ ]:
sentiment_gs = df_gs['text'].apply(lambda x: analyzer.polarity_scores(x))
df_gs = pd.concat([df_gs, sentiment_gs.apply(pd.Series)],1)

df_gs_pos = df_gs[df_gs['compound'] >= 0.05]
df_gs_neg = df_gs[df_gs['compound'] <= -0.05]
df_gs_neu = df_gs[( df_gs['compound'] < 0.05 ) & ( df_gs['compound'] > -0.05 )]

In [ ]:
dtm_tfidf = tfidf_vectorizer.fit_transform(df_gs_pos['text'].values)
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [ ]:
dtm_tfidf = tfidf_vectorizer.fit_transform(df_gs_neg['text'].values)
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [ ]:
dtm_tfidf = tfidf_vectorizer.fit_transform(df_gs_neu['text'].values)
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [ ]:
print(df_gs_pos.shape)
print(df_gs_neg.shape)
print(df_gs_neu.shape)

In [ ]:
sentiment_ms = df_ms['text'].apply(lambda x: analyzer.polarity_scores(x))
df_ms = pd.concat([df_ms, sentiment_ms.apply(pd.Series)],1)

df_ms_pos = df_ms[df_ms['compound'] >= 0.05]
df_ms_neg = df_ms[df_ms['compound'] <= -0.05]
df_ms_neu = df_ms[( df_ms['compound'] < 0.05 ) & ( df_ms['compound'] > -0.05 )]

In [ ]:
print(df_ms_pos.shape)
print(df_ms_neg.shape)
print(df_ms_neu.shape)

In [ ]:
dtm_tfidf = tfidf_vectorizer.fit_transform(df_ms_pos['text'].values)
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [ ]:
dtm_tfidf = tfidf_vectorizer.fit_transform(df_ms_neg['text'].values)
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [ ]:
dtm_tfidf = tfidf_vectorizer.fit_transform(df_ms_neu['text'].values)
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)